In [1]:
# Import statements
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
import os
import nltk
from nltk.stem import WordNetLemmatizer
import re
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression #Max entropy
from nltk import tokenize

In [2]:
#Process Yelp reviews to be similar to movie reviews
import unicodedata
reviewpath = "../Data/ExtraCredit/Yelp/all_reviews.txt"

old_neglines=[]
old_poslines=[]
#function to read all negative and positive lines into separate lists
f = open(reviewpath, mode="r",encoding='utf-8')
lines = f.read() #switched from readlines() to read the entire review
allrev =lines.split("]]]") #split each review. will need to process other brackets
allrev.pop() #removes the last item, which is empty
#For each review, isolate the text and put into positive or negative list
#depending on the associated rating. 1-3 stars = negative, and 4-5 stars = positive
for i in range(len(allrev)):
    currlines = allrev[i].splitlines()
    #Remove empty first line from reviews:
    if(i>0):
        currlines = currlines[1:]
    positive_review = False
    rating = int(currlines[1][0])
    if rating > 3:
        positive_review = True
    #Get review text: line 7 until end, minus 3 blank lines at the end
    currlines = currlines[7:-3]
    #print(currlines)
    currlines = " ".join(currlines)
    #Remove weird symbols like "\xa0":
    currlines = unicodedata.normalize("NFKD",currlines)
    #Add spaces between punctuation and words. Extra spaces don't matter.
    currlines = re.sub(r"([^\w\s])",r" \1 ", currlines)

    if positive_review:
        old_poslines.append(currlines)
    else:
        old_neglines.append(currlines)

#print(len(old_neglines)+len(old_poslines)) #number of reviews: 10391
#print(len(old_neglines)) #number of negative (1-3 star) reviews:1172+1358+1795 = 4325
#print(old_neglines[0])

In [3]:
#For Modification 3: Gets all words that are associated with an emotion.
emotion_path = "../Data/NRC-Sentiment-Emotion-Lexicons/NRC-Sentiment-Emotion-Lexicons/NRC-Emotion-Lexicon-v0.92/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt"
f = open(emotion_path, "r")
emotions = f.readlines()
emotion_words = set()
for line in emotions:
    emotion = line.split()
    #If the current word has an emotion associated with it, add it to the set
    if(emotion[2]=="1"):
        emotion_words.add(emotion[0])

In [4]:
#Preprocessing: lowercase and words only (remove punctuation, numbers)
from nltk.corpus import stopwords
#stopwords minus negation
stop_words = set(stopwords.words("english"))-set(["couldn't","wouldn","haven't","haven","aren","aren't","isn","isn't","ain","wouldn't","didn","didn't","doesn","doesn't","wasn't","don","don't","couldn","shouldn","shouldn't","hasn","hasn't","hadn","hadn't","not","won","won't",])
neg_words = "never|nothing|nowhere|none|havent|hasnt|hadnt|cant|couldnt|shouldnt|wont|wouldnt|dont|doesnt|didnt|isnt|arent|aint|no$|not$"

def preprocess(listlines):
    processed = []
    for line in listlines:
        #lowercase all reviews:
        newline = line.lower()
        
        ########### Modification 3: Remove sentences with no emotion words ##########
        # sentences=nltk.sent_tokenize(newline)
        # indices = [] #indices of sentences without emotion words
        # for i in range(len(sentences)):
        #         templine = sentences[i].split()
        #         #Doesn't deal with length-1 sentences (punctuation)
        #         if len(sentences[i])>1:
        #                 emo = False #tracks if any emotion words appear in sentence
        #                 for word in templine:
        #                         #lemmatize each word to check if lemma in emotion words
        #                         lemmatizer = WordNetLemmatizer()
        #                         lemword = lemmatizer.lemmatize(word)
        #                         if lemword in emotion_words:
        #                                 emo = True
        #                                 break
        #                 if emo == False:
        #                         #print(sentences[i])
        #                         indices.append(i)
        # #delete sentences
        # for i in reversed(indices):
        #         del sentences[i]
        # #update newline to reflect changes
        # newline=" ".join(sentences)
        ########################################################################

        #remove stopwords
        newline = newline.split()
        newline = [word for word in newline if not word in stop_words]
        newline = ' '.join(newline)

        # ########### Modification 2: Mark words following negative words ##########
        punct = "[.:;!?]"
        newline = re.sub(r"[^a-zA-z.:;!?\s]",'',newline) #keep clause-level punctuation first
        newline = newline.split()
        #iterate through tokens and mark words. Note clause-level punctuation is its own token
        count = 0
        while count < len(newline):
            #if current word is a negation, process
            if re.search(neg_words,newline[count]) is not None:
                count+=1
                #mark all words until clause-level punctuation reached
                while count < len(newline) and re.search(punct,newline[count]) is None:
                    newline[count] = newline[count]+"_NEG"
                    count+=1
            else:
                count+=1
        newline = ' '.join(newline)
        ########################################################################        
        #remove non-alphabet characters, keeping underscore.
        newline = re.sub(r"[^a-zA-z\s_]",'',newline) 
        ########### Modification 1: Replace negation with "not" ##########
        newline = re.sub(neg_words,"not",newline)
        ########################################################################
        #print(newline)
        processed.append(newline)
    return processed
neglines = preprocess(old_neglines)
poslines = preprocess(old_poslines)

In [5]:
#label each line with "positive" or "negative"
def labellines(listlines, sentiment):
    labeled = []
    for line in listlines:
        curr_line = []
        curr_line.append(line)
        curr_line.append(sentiment)
        labeled.append(curr_line)
    return labeled
neglabeled=labellines(neglines,"Negative")
poslabeled=labellines(poslines,"Positive")
lines = neglabeled+poslabeled

In [6]:
df = pd.DataFrame(lines, columns=["word","sentiment"])
print(df.head(10))

                                                word sentiment
0  ordered kung pao chicken came pieces chicken c...  Negative
1  food good service awful  placed order online w...  Negative
2  seeking decent chinese take delivery restauran...  Negative
3  took minutes food get forgot spring roll  hung...  Negative
4  disappointed entre e purchase  vegetables hard...  Negative
5  not vietnamese_NEG restaurant_NEG  friend orde...  Negative
6  place awesome jacked prices  dollars spring ro...  Negative
7  lived champaign years want love pekara support...  Negative
8  travelling stopped breakfast  organic eggs ome...  Negative
9  worst crepe        european cafe don know make...  Negative


In [7]:
labels = df["sentiment"]
data = df.drop(columns=["sentiment"])
X = data["word"]

In [8]:
#From demo:
# Define a function to take as input training and testing vectors and labels
# Allow this to be extensible to let multiple classifiers be used here
def buildClassifiers(clf, X_train, X_test, y_train, y_test):

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # if you get a zero-devision warning message, you can supress it by setting zero_division=0
    f1 = f1_score(y_test, y_pred, average="macro", zero_division=0)
    precision = precision_score(y_test, y_pred, average="macro", zero_division=0)
    recall = recall_score(y_test, y_pred, average="macro", zero_division=0)
    accuracy = accuracy_score(y_test, y_pred)
    return f1, precision, recall, accuracy

In [9]:
# Construct the classifiers at hand prior to folding the data through them
names = ['Naive_Bayes', 'Decision_Tree','Max_Entropy']#,'Support_Vector_Machines']
classifiers = [MultinomialNB(), 
              DecisionTreeClassifier(random_state=0, criterion='gini'), LogisticRegression(random_state=0,max_iter=1000)]#, SVC(kernel='linear')]
# names = ['Max_Entropy']
# classifiers = [LogisticRegression(random_state=0,max_iter=1000)]
# Try different classifiers: https://scikit-learn.org/stable/supervised_learning.html#supervised-learning
for name, clf in zip(names, classifiers):

    #print('Now classifying', name)

    # Fold the data 5 times
    kf = KFold(n_splits = 5, shuffle=True, random_state=0) #changed to not need shuffle
    foldCounter = 0
    aList, bList, cList, dList = list(), list(), list(), list()
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        #vectorize here:
        #vectorizer = CountVectorizer() #comment out if using tfidf
        ######################Modification 4: Use tfidf#####################
        vectorizer = TfidfVectorizer()
        ####################################################################
        X_train_vect = vectorizer.fit_transform(X_train).toarray()
        X_test_vect = vectorizer.transform(X_test).toarray()
        f1, precision, recall, accuracy = buildClassifiers(clf, X_train_vect, X_test_vect, y_train, y_test)
        aList.append(f1)
        bList.append(precision)
        cList.append(recall)
        dList.append(accuracy)
    print("Average F1 for {}:\t\t".format(name), np.mean(aList))
    print("Average Precision for {}:\t".format(name), np.mean(bList))
    print("Average Recall for {}:\t".format(name), np.mean(cList))
    print("Average Accuracy for {}:\t".format(name), np.mean(dList))

Average F1 for Naive_Bayes:		 0.6748063625244479
Average Precision for Naive_Bayes:	 0.8224607136285511
Average Recall for Naive_Bayes:	 0.6812179906757473
Average Accuracy for Naive_Bayes:	 0.7323623512266438
Average F1 for Decision_Tree:		 0.718858070191166
Average Precision for Decision_Tree:	 0.7180758662104312
Average Recall for Decision_Tree:	 0.7233627758116061
Average Accuracy for Decision_Tree:	 0.7228356714400987
Average F1 for Max_Entropy:		 0.822543516304471
Average Precision for Max_Entropy:	 0.8208692991369885
Average Recall for Max_Entropy:	 0.8252525471104615
Average Accuracy for Max_Entropy:	 0.8262909585334993
